In [117]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
This is a sample RNN code.
Using this original code, you could already get a 0.63 accuracy.
Please make some changes instead of submitting the original code.
If you have some questions about the code or data,
please contact M.M. Kuang(kuangmeng@hku.hk) or Y.Q. Deng(yqdeng@cs.hku.hk)
"""
import time
import sys
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

map_fn = tf.map_fn

In [118]:
#new
def str_2_list(data_list):
    ret_list = []
    for i in range(len(data_list)):
        tmp_list = data_list[i].strip().split(" ")
        tmp_ret_list = [int(tmp_list[7][0]),int(tmp_list[6]),int(tmp_list[5]),int(tmp_list[4]),int(tmp_list[3]),int(tmp_list[2]),int(tmp_list[1]),int(tmp_list[0][1])] #changed
        ret_list.append(tmp_ret_list)
    return ret_list

# Prepare Data(Training and Testing)
filename = "data.txt"
a_list = []
b_list = []
c_list = []


In [119]:
with open(filename, "r") as file:
    filein = file.read().splitlines()
    for item in filein:
        tmp_list = item.strip().split(",")
        a_list.append(tmp_list[0])
        b_list.append(tmp_list[1])
        c_list.append(tmp_list[2])
a_list = str_2_list(a_list)
b_list = str_2_list(b_list)
c_list = str_2_list(c_list)

In [259]:
# Define the dataflow graph
time_steps = 8        # time steps which is the same as the length of the bit-string
input_dim = 2         # number of units in the input layer
hidden_dim = 24      # number of units in the hidden layer
output_dim = 1        # number of units in the output layer
binary_dim = 8
largest_number = pow(2, binary_dim)
TINY          = 1e-6    # to avoid NaNs in logs

In [260]:
tf.reset_default_graph()
# input inputs and target ouput Y
inputs = tf.placeholder(tf.float32, [time_steps, None, input_dim], name='x')
outputs = tf.placeholder(tf.float32, [time_steps, None, output_dim], name='y')

# define the RNN cell: can be simple cell, LSTM or GRU
cell = tf.contrib.rnn.BasicLSTMCell(hidden_dim, state_is_tuple=True)
# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_dim, activation=tf.nn.tanh)

batch_size    = tf.shape(inputs)[1]
initial_state = cell.zero_state(batch_size, tf.float32)

# Given inputs (time, batch, input_size) outputs a tuple
#  - outputs: (time, batch, output_size)  [do not mistake with output_dim]
#  - states:  (time, batch, hidden_size)
rnn_outputs, rnn_states = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, time_major=True)

# project output from rnn output size to output_dim. Sometimes it is worth adding
# an extra layer here.
final_projection = lambda x: layers.linear(x, num_outputs=output_dim, activation_fn=tf.nn.sigmoid)

# apply projection to every timestep.
predicted_outputs = map_fn(final_projection, rnn_outputs)

# # minimize error, using ADAM as weight update rule
error = -(outputs * tf.log(predicted_outputs + TINY) + (1.0 - outputs) * tf.log(1.0 - predicted_outputs + TINY))
error = tf.reduce_sum(error)

learning_rate = 0.06
# global_step = tf.Variable(0, trainable=False)
decayed_lr = tf.train.exponential_decay(learning_rate, global_step, 100000, 0.99, staircase=True)
# train_fn = tf.train.AdamOptimizer(decayed_lr).minimize(error, global_step=global_step)
train_fn = tf.train.AdamOptimizer(learning_rate).minimize(error)
accuracy = tf.reduce_mean(tf.cast(tf.abs(outputs - predicted_outputs) < 0.5, tf.float32))

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Number of data for training and testing
#Please remember the total number is 5000
num4train = 3000
num4valid = 1000
num4test = 1000

In [261]:
# train
batch_size = 500
early_stopping_tolerance = 2
early_stopping_counter = 0
min_valid_error = sys.maxsize


valid_a = np.transpose([a_list[num4train: num4train + num4valid]])
valid_b = np.transpose([b_list[num4train: num4train + num4valid]])
valid_y = np.transpose([c_list[num4train: num4train + num4valid]])
valid_x = np.concatenate((valid_a,valid_b), axis=2)

num_of_epochs = 0
start_time = time.time()
while True:
    num_of_epochs = num_of_epochs + 1
    train_error = 0
    ITERATIONS_PER_EPOCH = int(num4train/batch_size)
    for j in range(ITERATIONS_PER_EPOCH):
        a = np.transpose([a_list[j*batch_size : (j+1)*batch_size]])
        b = np.transpose([b_list[j*batch_size : (j+1)*batch_size]])
        y = np.transpose([c_list[j*batch_size : (j+1)*batch_size]])
        x = np.concatenate((a,b), axis=2)

        train_error += sess.run([error, train_fn], {
            inputs: x,
            outputs: y,
        })[0]
    train_error /= ITERATIONS_PER_EPOCH
    valid_accuracy = sess.run(accuracy, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    valid_error = sess.run(error, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    print("Epoch ", num_of_epochs, ", train err: ", train_error, "valid err: ", valid_error, ", valid acc: ", valid_accuracy * 100.0)
    #early stopping
    if valid_error < 1:
            #early stop
            break
            
#     if valid_error < min_valid_error:
#         min_valid_error = valid_error
#         early_stopping_counter = 0
#     else:
#         early_stopping_counter = early_stopping_counter + 1
#         print("early_stopping_counter: " + str(early_stopping_counter))
#         if early_stopping_counter >= early_stopping_tolerance:
#             #early stop
#             break
end_time = time.time()

remain_result = []
print("Training Finished")
print("Training time: ", end_time - start_time)

Epoch  1 , train err:  2756.144734700521 valid err:  5549.352 , valid acc:  49.33750033378601
Epoch  2 , train err:  2747.355224609375 valid err:  5485.674 , valid acc:  46.26249969005585
Epoch  3 , train err:  2678.2693684895835 valid err:  5350.381 , valid acc:  62.08750009536743
Epoch  4 , train err:  2567.799519856771 valid err:  5091.951 , valid acc:  77.01249718666077
Epoch  5 , train err:  2416.438517252604 valid err:  4583.998 , valid acc:  79.78749871253967
Epoch  6 , train err:  2138.5991007486978 valid err:  3807.6187 , valid acc:  78.87499928474426
Epoch  7 , train err:  1912.4672444661458 valid err:  3279.524 , valid acc:  82.7750027179718
Epoch  8 , train err:  1630.6675211588542 valid err:  2795.4263 , valid acc:  87.66250014305115
Epoch  9 , train err:  1422.3919881184895 valid err:  2447.7822 , valid acc:  86.73750162124634
Epoch  10 , train err:  1174.019063313802 valid err:  1939.4226 , valid acc:  87.5124990940094
Epoch  11 , train err:  897.9648234049479 valid err:

Epoch  99 , train err:  0.9893107265233994 valid err:  1.0223092 , valid acc:  100.0
Epoch  100 , train err:  0.970686470468839 valid err:  1.0013353 , valid acc:  100.0
Epoch  101 , train err:  0.9525251984596252 valid err:  0.9809315 , valid acc:  100.0
Training Finished
Training time:  10.60700798034668


In [262]:
remain_result = []
average_error = 0
#Test
for i in range(num4train + num4valid + 1, num4train + num4valid + num4test):
    a = np.transpose([[a_list[i]]])
    b = np.transpose([[b_list[i]]])
    y = np.transpose([[c_list[i]]])
    x = np.concatenate((a,b), axis=2)

    # get predicted value
    [_probs, _error] = sess.run([predicted_outputs, error], {inputs: x, outputs: y})
    probs = np.array(_probs).reshape([8])
    prediction = np.array([1 if p >= 0.5 else 0 for p in probs]).reshape([8])
    # Save the result
    disired_output = y.reshape([binary_dim])
    remain_result.append([prediction, disired_output])

    # calculate error
    y = y.reshape(binary_dim)
    probs = probs.reshape(binary_dim)
    error_value = np.sum(np.absolute(y - probs))
    
    #print the prediction, the right y and the error.
    print("---------------")
    print(prediction)
    print(disired_output)
    print(error_value)
    print("---------------")
    print()

average_error = average_error/num4test
print(average_error)

---------------
[0 0 0 1 0 1 0 1]
[0 0 0 1 0 1 0 1]
0.0008378698057640577
---------------

---------------
[0 0 1 1 1 0 0 1]
[0 0 1 1 1 0 0 1]
0.0008420162957918365
---------------

---------------
[0 1 1 0 1 0 0 1]
[0 1 1 0 1 0 0 1]
0.004080934766534483
---------------

---------------
[0 0 0 0 1 0 0 1]
[0 0 0 0 1 0 0 1]
0.00029254573564685415
---------------

---------------
[0 0 1 0 0 0 0 1]
[0 0 1 0 0 0 0 1]
0.0016617223727735109
---------------

---------------
[0 1 1 1 1 1 1 0]
[0 1 1 1 1 1 1 0]
0.0003990115728811361
---------------

---------------
[0 0 0 1 1 1 1 0]
[0 0 0 1 1 1 1 0]
0.0003745132617041236
---------------

---------------
[0 1 0 0 1 1 1 0]
[0 1 0 0 1 1 1 0]
0.0004650306964322226
---------------

---------------
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
0.00012865047722243617
---------------

---------------
[0 0 0 1 0 1 1 1]
[0 0 0 1 0 1 1 1]
0.00022979107779974584
---------------

---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.00025887234733090736
-------------

---------------
[0 1 0 0 0 0 1 1]
[0 1 0 0 0 0 1 1]
0.0017302273572568083
---------------

---------------
[0 0 1 1 1 1 0 1]
[0 0 1 1 1 1 0 1]
0.0007328233514272142
---------------

---------------
[0 1 1 0 1 1 0 1]
[0 1 1 0 1 1 0 1]
0.005042595032136887
---------------

---------------
[0 0 0 0 1 1 0 1]
[0 0 0 0 1 1 0 1]
0.00023403017803502735
---------------

---------------
[0 0 1 0 0 1 0 1]
[0 0 1 0 0 1 0 1]
0.0017764117646947852
---------------

---------------
[0 1 1 1 1 0 0 1]
[0 1 1 1 1 0 0 1]
0.003156552542350255
---------------

---------------
[0 0 0 1 1 0 0 1]
[0 0 0 1 1 0 0 1]
0.0011587530443648575
---------------

---------------
[0 1 0 0 1 0 0 1]
[0 1 0 0 1 0 0 1]
0.0018669342334760586
---------------

---------------
[0 0 1 1 0 0 0 1]
[0 0 1 1 0 0 0 1]
0.0012668191666307393
---------------

---------------
[0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1]
0.00011736450596799841
---------------

---------------
[0 1 0 1 1 1 1 0]
[0 1 0 1 1 1 1 0]
0.0003767219568544533
---------------


---------------
[0 1 1 0 1 0 0 1]
[0 1 1 0 1 0 0 1]
0.004080934766534483
---------------

---------------
[0 0 0 0 1 0 0 1]
[0 0 0 0 1 0 0 1]
0.00029254573564685415
---------------

---------------
[0 1 0 1 0 0 0 1]
[0 1 0 1 0 0 0 1]
0.003753026667254744
---------------

---------------
[0 0 1 0 0 0 0 1]
[0 0 1 0 0 0 0 1]
0.0016617223727735109
---------------

---------------
[0 1 1 1 1 1 1 0]
[0 1 1 1 1 1 1 0]
0.0003990115728811361
---------------

---------------
[0 1 0 0 1 1 1 0]
[0 1 0 0 1 1 1 0]
0.0004650306964322226
---------------

---------------
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
0.00012865047722243617
---------------

---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0002733610599534586
---------------

---------------
[0 0 0 1 0 1 1 1]
[0 0 0 1 0 1 1 1]
0.00022979107779974584
---------------

---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.00025887234733090736
---------------

---------------
[0 0 1 0 0 0 1 1]
[0 0 1 0 0 0 1 1]
0.00032322387323802104
-------------

---------------
[0 0 0 0 1 1 1 1]
[0 0 0 0 1 1 1 1]
0.00019661117948999163
---------------

---------------
[0 0 1 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
0.00029452428771037376
---------------

---------------
[0 0 0 1 1 0 1 1]
[0 0 0 1 1 0 1 1]
0.000664881003103801
---------------

---------------
[0 0 0 0 0 0 1 1]
[0 0 0 0 0 0 1 1]
0.00012095521014998667
---------------

---------------
[0 0 1 0 1 1 0 1]
[0 0 1 0 1 1 0 1]
0.000944364073802717
---------------

---------------
[0 0 0 1 0 1 0 1]
[0 0 0 1 0 1 0 1]
0.0008378698057640577
---------------

---------------
[0 0 1 1 1 0 0 1]
[0 0 1 1 1 0 0 1]
0.0008420162957918365
---------------

---------------
[0 1 1 0 1 0 0 1]
[0 1 1 0 1 0 0 1]
0.004080934766534483
---------------

---------------
[0 1 0 1 0 0 0 1]
[0 1 0 1 0 0 0 1]
0.003753026667254744
---------------

---------------
[0 0 1 0 0 0 0 1]
[0 0 1 0 0 0 0 1]
0.0016617223727735109
---------------

---------------
[0 1 1 1 1 1 1 0]
[0 1 1 1 1 1 1 0]
0.0003990115728811361
---------------



---------------
[0 0 1 1 0 1 1 1]
[0 0 1 1 0 1 1 1]
0.0007753661084279884
---------------

---------------
[0 0 0 0 0 1 1 1]
[0 0 0 0 0 1 1 1]
0.0001313609736826038
---------------

---------------
[0 0 0 1 0 0 1 1]
[0 0 0 1 0 0 1 1]
0.0003188383880114998
---------------

---------------
[0 0 1 1 1 1 0 1]
[0 0 1 1 1 1 0 1]
0.00041989896635641344
---------------

---------------
[0 0 1 1 1 1 1 1]
[0 0 1 1 1 1 1 1]
0.00021429641128634103
---------------

---------------
[0 0 0 0 1 1 1 1]
[0 0 0 0 1 1 1 1]
0.00025812317289819475
---------------

---------------
[0 0 1 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
0.00027223215056437766
---------------

---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0003082893817918375
---------------

---------------
[0 1 1 1 0 1 1 1]
[0 1 1 1 0 1 1 1]
0.00040959484613267705
---------------

---------------
[0 0 0 1 0 1 1 1]
[0 0 0 1 0 1 1 1]
0.0002166780559491599
---------------

---------------
[0 1 0 0 0 1 1 1]
[0 1 0 0 0 1 1 1]
0.0006920871473994339
-----------

---------------
[0 0 1 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
0.00027223215056437766
---------------

---------------
[0 1 0 1 1 1 1 1]
[0 1 0 1 1 1 1 1]
0.0005037589544372167
---------------

---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0003082893817918375
---------------

---------------
[0 0 0 1 0 1 1 1]
[0 0 0 1 0 1 1 1]
0.0002166780559491599
---------------

---------------
[0 1 0 0 0 1 1 1]
[0 1 0 0 0 1 1 1]
0.0006920871473994339
---------------

---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.00027150853202329017
---------------

---------------
[0 1 1 0 1 0 1 1]
[0 1 1 0 1 0 1 1]
0.0008602563466411084
---------------

---------------
[0 0 0 0 1 0 1 1]
[0 0 0 0 1 0 1 1]
0.00017942479280463886
---------------

---------------
[0 0 1 0 0 0 1 1]
[0 0 1 0 0 0 1 1]
0.000420260234932357
---------------

---------------
[0 1 1 1 1 1 0 1]
[0 1 1 1 1 1 0 1]
0.0004891113712801598
---------------

---------------
[0 0 0 1 1 1 0 1]
[0 0 0 1 1 1 0 1]
0.0002930152386397822
--------------

---------------
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
0.00012865047722243617
---------------

---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0002733610599534586
---------------

---------------
[0 0 0 1 0 1 1 1]
[0 0 0 1 0 1 1 1]
0.00022979107779974584
---------------

---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.00025887234733090736
---------------

---------------
[0 0 0 0 1 0 1 1]
[0 0 0 0 1 0 1 1]
0.00020243218568793964
---------------

---------------
[0 0 0 1 1 1 0 1]
[0 0 0 1 1 1 0 1]
0.00027978300749964546
---------------

---------------
[0 0 1 1 0 1 0 1]
[0 0 1 1 0 1 0 1]
0.0006758514618923073
---------------

---------------
[0 0 0 0 0 1 0 1]
[0 0 0 0 0 1 0 1]
0.00015136890397116076
---------------

---------------
[0 0 1 0 1 0 0 1]
[0 0 1 0 1 0 0 1]
0.00037007387436460704
---------------

---------------
[0 0 0 1 1 1 1 1]
[0 0 0 1 1 1 1 1]
0.00021449298765219282
---------------

---------------
[0 0 0 0 0 1 1 1]
[0 0 0 0 0 1 1 1]
0.0001313609736826038
--------

---------------
[0 0 1 1 0 0 1 1]
[0 0 1 1 0 0 1 1]
0.0017207730743393768
---------------

---------------
[0 0 1 0 1 1 0 1]
[0 0 1 0 1 1 0 1]
0.000944364073802717
---------------

---------------
[0 0 0 1 0 1 0 1]
[0 0 0 1 0 1 0 1]
0.0008378698057640577
---------------

---------------
[0 0 1 1 1 0 0 1]
[0 0 1 1 1 0 0 1]
0.0008420162957918365
---------------

---------------
[0 1 1 0 1 0 0 1]
[0 1 1 0 1 0 0 1]
0.004080934766534483
---------------

---------------
[0 0 0 0 1 0 0 1]
[0 0 0 0 1 0 0 1]
0.00029254573564685415
---------------

---------------
[0 0 1 0 0 0 0 1]
[0 0 1 0 0 0 0 1]
0.0016617223727735109
---------------

---------------
[0 1 1 1 1 1 1 0]
[0 1 1 1 1 1 1 0]
0.0003990115728811361
---------------

---------------
[0 0 0 1 1 1 1 0]
[0 0 0 1 1 1 1 0]
0.0003745132617041236
---------------

---------------
[0 1 0 0 1 1 1 0]
[0 1 0 0 1 1 1 0]
0.0004650306964322226
---------------

---------------
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
0.00012865047722243617
---------------


In [263]:
# Get the total accuracy (Please don't change this part)
accuracy = 0
for i in range(len(remain_result)):
    len_ = len(remain_result[i][0])
    tmp_num = 0
    for j in range(len_):
        if remain_result[i][0][j] == remain_result[i][1][j]:
            tmp_num += 1
    accuracy += tmp_num / len_

accuracy /= len(remain_result)

print("Accuracy: %.4f"%(accuracy))

Accuracy: 1.0000


In [149]:
sess.close()